### TODO
- [ ] Add more data
- [ ] Add more issues from other repositories
- [ ] Add closed issues from nextjs

In [28]:
import pandas as pd
import mysql.connector
import requests as req
from dotenv import load_dotenv
from helper import execute_query, iso_to_datetime, github_api
import os
load_dotenv()


True

In [29]:
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
pd.set_option('display.max_columns', None)


In [30]:
cnx = mysql.connector.connect(user='root', password='Rvega2015',
                              host='127.0.0.1',
                              database='github')

In [31]:
df = pd.read_json('../json/language/all.json')
df.sort_values(by='stargazers_count', ascending=False, inplace=True)
df = df[df['open_issues'] > 1000]

df.head(1)

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,forks_url,keys_url,collaborators_url,teams_url,hooks_url,issue_events_url,events_url,assignees_url,branches_url,tags_url,blobs_url,git_tags_url,git_refs_url,trees_url,statuses_url,languages_url,stargazers_url,contributors_url,subscribers_url,subscription_url,commits_url,git_commits_url,comments_url,issue_comment_url,contents_url,compare_url,merges_url,archive_url,downloads_url,issues_url,pulls_url,milestones_url,notifications_url,labels_url,releases_url,deployments_url,created_at,updated_at,pushed_at,git_url,ssh_url,clone_url,svn_url,homepage,size,stargazers_count,watchers_count,language,has_issues,has_projects,has_downloads,has_wiki,has_pages,forks_count,mirror_url,archived,disabled,open_issues_count,license,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,score
200,70107786,MDEwOlJlcG9zaXRvcnk3MDEwNzc4Ng==,next.js,vercel/next.js,False,"{'login': 'vercel', 'id': 14985020, 'node_id':...",https://github.com/vercel/next.js,The React Framework,False,https://api.github.com/repos/vercel/next.js,https://api.github.com/repos/vercel/next.js/forks,https://api.github.com/repos/vercel/next.js/ke...,https://api.github.com/repos/vercel/next.js/co...,https://api.github.com/repos/vercel/next.js/teams,https://api.github.com/repos/vercel/next.js/hooks,https://api.github.com/repos/vercel/next.js/is...,https://api.github.com/repos/vercel/next.js/ev...,https://api.github.com/repos/vercel/next.js/as...,https://api.github.com/repos/vercel/next.js/br...,https://api.github.com/repos/vercel/next.js/tags,https://api.github.com/repos/vercel/next.js/gi...,https://api.github.com/repos/vercel/next.js/gi...,https://api.github.com/repos/vercel/next.js/gi...,https://api.github.com/repos/vercel/next.js/gi...,https://api.github.com/repos/vercel/next.js/st...,https://api.github.com/repos/vercel/next.js/la...,https://api.github.com/repos/vercel/next.js/st...,https://api.github.com/repos/vercel/next.js/co...,https://api.github.com/repos/vercel/next.js/su...,https://api.github.com/repos/vercel/next.js/su...,https://api.github.com/repos/vercel/next.js/co...,https://api.github.com/repos/vercel/next.js/gi...,https://api.github.com/repos/vercel/next.js/co...,https://api.github.com/repos/vercel/next.js/is...,https://api.github.com/repos/vercel/next.js/co...,https://api.github.com/repos/vercel/next.js/co...,https://api.github.com/repos/vercel/next.js/me...,https://api.github.com/repos/vercel/next.js/{a...,https://api.github.com/repos/vercel/next.js/do...,https://api.github.com/repos/vercel/next.js/is...,https://api.github.com/repos/vercel/next.js/pu...,https://api.github.com/repos/vercel/next.js/mi...,https://api.github.com/repos/vercel/next.js/no...,https://api.github.com/repos/vercel/next.js/la...,https://api.github.com/repos/vercel/next.js/re...,https://api.github.com/repos/vercel/next.js/de...,2016-10-05 23:32:51+00:00,2022-10-27 21:55:35+00:00,2022-10-27 22:13:56+00:00,git://github.com/vercel/next.js.git,git@github.com:vercel/next.js.git,https://github.com/vercel/next.js.git,https://github.com/vercel/next.js,https://nextjs.org,1786033,94858,94858,JavaScript,True,False,True,False,False,20850,None,False,False,1437,"{'key': 'mit', 'name': 'MIT License', 'spdx_id...",True,False,False,"[blog, browser, compiler, components, hybrid, ...",public,20850,1437,94858,canary,1


In [32]:
def add_developer(developer) -> None:
        query = "INSERT INTO developers (id, login, html_url, type) VALUES (%s,%s,%s,%s)"
        values = (developer['id'], developer['login'], developer['html_url'], developer['type'])
        execute_query(type='INSERT', query=query, values=values)


def developers_to_db():
    rows = df.shape[0]
    for i in range(rows):
        row = df.iloc[i]
        developer = row['owner']
        add_developer(developer)


In [33]:
important_columns = ['id', 'full_name', 'owner', 'size', 'watchers_count', 'stargazers_count', 'description',
                     'html_url', 'name', 'language', 'forks_count', 'open_issues_count', 'created_at', 'updated_at']


def get_important_fields(row) -> list:
    important_fields = []
    for column in important_columns:
        if column in list(row.keys()):
            if (column == 'owner'):
                important_fields.append(row[column]['id'])
            else:
                important_fields.append(row[column])
    return important_fields


def repositories_to_db():
    rows = df.shape[0]
    for i in range(rows):
        row = df.iloc[i].to_dict()
        important_fields = get_important_fields(row)
        query = "INSERT INTO repositories (id, full_name, owner_id, size, watchers_count, stargazers_count, description, url, name, language, forks_count, open_issues_count, created_at, pushed_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        execute_query(type='INSERT', query=query, values=important_fields)



In [35]:
def check_if_user_exist(user):
    query = "SELECT * FROM developers WHERE id = %s"
    values = (user['id'],)
    result = execute_query(type='SELECT', query=query, values=values)
    return len(result) > 0


def issues_to_db():
    repositories = execute_query(type='SELECT', query="SELECT id, full_name FROM repositories", values=None)
    for repo in repositories:
        full_name = repo[1]
        repo_id = repo[0]
        issues = github_api('issues', full_name, {
                            'state': 'all', 'per_page': 100})

        for issue in issues:
            created_at = iso_to_datetime(issue['created_at'])
            closed_at = iso_to_datetime(
                issue['closed_at']) if issue['closed_at'] else None
            user_exist = check_if_user_exist(issue['user'])
            if not user_exist:
                add_developer(issue['user'])
            execute_query(
                type='INSERT',
                query="INSERT INTO issues (id, title, body, created_at, closed_at, state, repo_id, user_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                values=(issue['id'], issue['title'], issue['body'], created_at, closed_at, issue['state'], repo_id, issue['user']['id']))


issues_to_db()


INSERT DONE
1062 (23000): Duplicate entry '1442350478' for key 'issues.PRIMARY'
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT DONE
INSERT D